In [4]:
import pandas as pd
import geopandas as gpd
import folium
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objs as go
from ipywidgets import Output

# Load map data and housing price data
coverage_data  = pd.read_excel('group_coverage.xlsx')
geo_df = gpd.read_file('capital_region_municipalities.geojson')
house_prices = pd.read_excel('house_price.xlsx')


# Initialize weights and sliders
facilities = ['company', 'eat', 'gym', 'hospital', 'park', 'school', 'shop', 'station']
weights = {fac: 0 for fac in facilities}
sliders = {fac: widgets.FloatSlider(value=0, min=0, max=1, step=0.01, description=f'{fac.capitalize()}:') for fac in weights}
total_weight_label = widgets.Label('Total Weight: 0.00')
button = widgets.Button(description="Start Update", disabled=True)

# Create output containers
chart_container = widgets.Output()
map_container = widgets.Output()

# Function to update total weight and check constraints
def update_total_weight(change):
    total_weight = sum(slider.value for slider in sliders.values())
    total_weight_label.value = f"Total Weight: {total_weight:.2f}"
    button.disabled = not np.isclose(total_weight, 1.0)

# Connect sliders to the weight updating function
for slider in sliders.values():
    slider.observe(update_total_weight, 'value')

# Function to update visuals on button click
def on_button_clicked(b):
    update_visuals()

button.on_click(on_button_clicked)

# Function to update map and chart
def update_visuals():
    merged_df = create_map()
    top_municipalities = merged_df.nsmallest(3, 'rank')['name_gn_spell_spellofna_text'].tolist()
    create_chart(top_municipalities)

def create_map():
    merged_df = geo_df.merge(coverage_data, on='name_gn_spell_spellofna_text')
    for slider in sliders.values():
        weights[slider.description.strip(':').lower()] = slider.value
    for fac, weight in weights.items():
        merged_df[fac + '_score'] = merged_df[fac + '_coverage'] * weight
    merged_df['total_score'] = merged_df[[f"{fac}_score" for fac in weights]].sum(axis=1)

    mean_score = merged_df['total_score'].mean()
    std_dev = merged_df['total_score'].std()
    merged_df['z_score'] = (merged_df['total_score'] - mean_score) / std_dev
    merged_df['normalized_score'] = round(50 + (merged_df['z_score'] * 10))
    merged_df['normalized_score'] = merged_df['normalized_score'].clip(lower=0, upper=100)
    merged_df['rank'] = merged_df['normalized_score'].rank(ascending=False, method='min')

    m = folium.Map(location=[55.6761, 12.5683], zoom_start=10)
    folium.GeoJson(
        merged_df,
        style_function=lambda feature: {
            'fillColor': '#CCCCCC' if feature['properties']['rank'] > 3 else '#FFEDA0' if feature['properties']['rank'] == 3 else '#FED976' if feature['properties']['rank'] == 2 else '#FEB24C',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['name_gn_spell_spellofna_text', 'normalized_score', 'rank'],
            aliases=['Municipality:', 'Score:', 'Rank:']
        )
    ).add_to(m)
    with map_container:
        clear_output(wait=True)
        display(m)
    return merged_df

# 预定义一组颜色用于不同的线条
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# 创建输出容器
chart_container = Output()

def create_chart(municipalities):
    with chart_container:
        clear_output(wait=True)
        data = []
        color_index = 0  # 初始化颜色索引

        # 为每个选中的市创建一条折线图
        for municipality in municipalities:
            # 获取完整数据
            full_data = house_prices.loc[municipality]
            # 分割实线与虚线数据
            solid_data = full_data[full_data.index < 2024]
            dashed_data = full_data[full_data.index >= 2024]
            
            # 实线部分
            trace_solid = go.Scatter(
                x=solid_data.index,
                y=solid_data,
                mode='lines+markers',
                name=municipality,
                line=dict(width=2, color=colors[color_index % len(colors)]),
                showlegend=False
                
            )
            data.append(trace_solid)
            
            # 虚线部分
            if not dashed_data.empty:
                trace_dashed = go.Scatter(
                    x=dashed_data.index,
                    y=dashed_data,
                    mode='lines+markers',
                    line=dict(width=2, color=colors[color_index % len(colors)], dash='dash'),
                    showlegend=False
                )
                data.append(trace_dashed)
            
            color_index += 1  # 更新颜色索引
        
        # 设置布局
        layout = go.Layout(
            title='Yearly Flat Price Trends for Top 3 Municipalities',
            xaxis=dict(title='Year'),
            yaxis=dict(title='Price (in thousand DKK)'),
            hovermode='closest',
            width=800,  # 设置图表宽度
            height=400   # 设置图表高度
        )
        
        # 生成图表
        fig = go.Figure(data=data, layout=layout)
        fig.show()

# Layout the widgets and output
control_box = widgets.VBox([total_weight_label] + list(sliders.values()) + [button])
layout = widgets.HBox([control_box, widgets.VBox([chart_container, map_container])])
display(layout)